In [1]:
# First import the common Python packages
import numpy as np
# Pandas is like spreadsheets in python
import pandas as pd
# Lets get it to display all pandas columns
pd.set_option('display.max_columns', None)
# This is for plotting
import matplotlib.pyplot as plt

# import pyMELTScalc and locate the MELTS files
import pyMELTScalc as M
import sys

sys.path.append(r"MELTS")

# MELTS tutorial Q3 & 4 - Crystallisation and decompression of basaltic magmas

In this notebook we'll show how pyMELTScalc can quickly perform multiple calculations, and investigate how the liquid-line-of-descent is linked to the conditions of magma storage.

First, we need to load in some data. For this example we'll use data from Isla Fernandina in the Galapagos Archipelago, where a series of melt inclusions and matrix glasses provide us with plenty of glass data to constain the magmatic evolution of the sub-volcanic system. The melt inclusion and matrix glass data is included in a single excel spreadsheet that we can load in using Pandas. We can also split this DataFrame into two. One containing just the melt inclusion data, the second including just the matrix glass data.

In [2]:
df = pd.read_excel('Fernandina_glass.xlsx')
df = df.fillna(0)

# split data based on the Group (Melt Inclusion or Matrix Glass)
MI = df.loc[df['Group'] == 'MI',:].reset_index(drop = True)
MG = df.loc[df['Group'] == 'MG',:].reset_index(drop = True)

MG.head()

One of the key things that we need to perform any calculations with MELTS is a starting composition. To start, let's use the most primitive (highest MgO) composition from the melt inclusion analyses. There are a few ways to find this composition, the way I like to do so (not necessarily the quickest) is to simply sort the melt inclusion DataFrame by the MgO contents and select the first row:

In [3]:
MI = MI.sort_values('MgO', ascending = False, ignore_index = True)
MI.head()

In [4]:
starting_comp = MI.loc[0]
# starting_comp['CO2'] = 0.5
# starting_comp['H2O'] = 0.4
starting_comp

## Q3 - Crystallisation under different conditions

### Q3.1 Using the starting composition selected above, simulate fractional crystallisation of magmas beneath Isla Fernandina at 500, 1000, 2000, and 4000 bars. Plot up the results.

These calculations should be run using rhyoliteMELTSv1.2.0 (we have some CO$_2$ in the starting composition). All models should start at the liquidus and end at 1100$^o$C. Run the calculations at 1 log unit below the FMQ buffer.

In [5]:
import os
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [6]:
Isobaric_Xtal = M.isobaric_crystallisation(Model = "MELTSv1.2.0",
                                           bulk = starting_comp,
                                           find_liquidus = True,
                                           P_bar = np.array([Enter Pressure Values]),
                                           T_end_C = 1100,
                                           dt_C = 2,
                                           fO2_buffer = "FMQ",
                                           fO2_offset = -1.0,
                                           Frac_solid = True,
                                           Frac_fluid = True,
                                           label = "pressure")

In [7]:
x_axis = 'MgO'
y_axis = ['Al2O3', 'FeOt', 'CaO']
f, a = plt.subplots(1,3, figsize = (12,4), sharex = True)
for i in range(len(y_axis)):
    a[i].plot(MI[x_axis], MI[y_axis[i]], 'ok', mfc = 'b', label = "Melt Inclusions")
    a[i].plot(MG[x_axis], MG[y_axis[i]], 'sk', mfc = 'y', label = "Matrix Glass")
    for r in Isobaric_Xtal:
        a[i].plot(Isobaric_Xtal[r]['All'][x_axis + '_Liq'], Isobaric_Xtal[r]['All'][y_axis[i] + '_Liq'],
                  '--', label = r)
        
a[2].legend()

### Q3.2 What model provides the best match to the data?

Melt inclusion volatile contents are influenced by numerous processes (e.g., diffusive H$^+$ loss, decrepitation, vapour bubble formation etc.). Gleeson et al. (2022) suggested that this melt inclusion likely contained ~0.4 wt% H$_2$O at the time of melt entrapment (and possibly up to 0.5 wt% CO$_2$). 

### Q3.3 Test the influence of melt volatile contents on the liquid-line-of-descent by changing the H$_2$O and CO$_2$ contents. How does this affect your results?

**HINT: use the commented lines above to overwrite the H$_2$O and CO$_2$ content of the initial melt composition**

We'll now move onto simulating the decompression of these magmas prior to/during eruption. The submarine matrix glasses were erupted under ~3000 m of water, so the assumption is that they did not loose any water during ascent towards the surface.

## Q4 - Degassing and decompression


When considering the decompression and ascent of the final magma prior to eruption, it's probably better to consider the matrix glass compositions, rather than the melt inclusions, probably provide a better estimate of the pre-eruptive magma composition. For these calculations let's simply calculate the average matrix glass composition and use that:

In [8]:
starting_comp = MG.apply(pd.to_numeric, errors='coerce').mean()
starting_comp

The Fernandina magmas we likely stored at ~3 kbar (3000 bars) prior to eruption. pyMELTScalc can use this information and the rhyoliteMELTSv1.2.0 model (MAGMASAT) to tell us how much CO$_2$ can dissolve in this magma at its liquidus.

In [9]:
T_Liq, H2O, CO2 = M.findCO2_multi(Model = "MELTSv1.2.0",
                                bulk = starting_comp.to_dict(),
                                fO2_buffer = "FMQ",
                                fO2_offset = -1.0,
                                P_bar = 3000)

### Q4.1 How much CO$_2$ can dissolve in the Fernandina magmas prior to eruption (during storage at ~3 kbar)?

In [10]:
CO2

Now we have an estimate of pressure, a pre-eruptive melt composition, and pre-eruptive volatile contents. Using that information it's straightforward to simulate isothermal decompression of this magma as it moves towards the surface.

In [11]:
Decompression_const_T = M.isothermal_decompression(Model = "MELTSv1.2.0",
                                                   bulk = starting_comp.to_dict(),
                                                   T_C = , # Enter the temperature to start at
                                                   P_start_bar = 3000,
                                                   P_end_bar = 300,
                                                   dp_bar = 25,
                                                   fO2_buffer = "FMQ",
                                                   fO2_offset = -1.0,
                                                   H2O_Liq = , # enter the initial melt H2O content
                                                   CO2_Liq = ) # enter the initial melt CO2 content

In [12]:
f, a = plt.subplots(1,3, figsize = (10,6), sharey = True)
a[0].plot(Decompression_const_T['Volume']['fluid1']/Decompression_const_T['Volume'].sum(axis = 1),
         Decompression_const_T['All']['P_bar']/1000, '-k')
a[0].set_ylim([3,0])
a[0].set_ylabel('Pressure (kbar)')
a[0].set_xlabel('Fluid volume fraction')

# convert outputs in wt% to mole fractions in the fluid
xH2O = (Decompression_const_T['fluid1']['H2O_fluid1']/18)/(Decompression_const_T['fluid1']['H2O_fluid1']/18 + 
                                                           Decompression_const_T['fluid1']['CO2_fluid1']/44)

a[1].plot(xH2O, Decompression_const_T['All']['P_bar']/1000, '-k', label = "X$_{H2O}$")
a[1].plot(1-xH2O, Decompression_const_T['All']['P_bar']/1000, ':k', label = "X$_{CO2}$")
a[1].set_ylim([3,0])
a[1].set_xlabel('Fluid mole fraction')
a[1].set_ylabel('Pressure (kbar)')
a[1].legend()

a[2].plot(Decompression_const_T['All']['H2O_Liq'], 
          Decompression_const_T['All']['P_bar']/1000, '-k', label = "H$_2$O")
a[2].plot(Decompression_const_T['All']['CO2_Liq'], 
          Decompression_const_T['All']['P_bar']/1000, 
          ':k', label = "CO$_2$")
a[2].set_ylim([3,0])
a[2].set_xlabel('Melt content (wt%)')
a[2].set_ylabel('Pressure (kbar)')
a[2].legend()

### Q4.2 What is the H$_2$O content of the melt phase at 300 bars (end of the model)?

### Q4.3 What is the maximum fluid volume fraction for this submarine eruption?
 
 **If you're waiting for others to finish, think about the following questions**
 - **This initial model represents closed system degassing (melt and fluid remain in equilibrium during decompression), if we add a command 'Frac_fluid = True' to the function above, we can instead simulate open system degassing (fluid separates from the melt as it forms). Try implementing this change and consider the same questions.**
 - **What problems might be associated with the isothermal assumption used in this calculation? What other constraints could we impose on the thermal properties of the decompressing magma?**          

In [13]:
Decompression_const_T['All']['H2O_Liq']

In [14]:
Decompression_const_T['liquid1']